## Imports

In [1]:
import sys

sys.path.append("..")

import convnext

In [2]:
from tensorflow import keras
import tensorflow as tf

from imutils import paths
import json
import os

## Constants

In [3]:
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 256
IMAGE_SIZE = 224
BASE_WEIGHTS_PATH = "https://storage.googleapis.com/convnext-tf/keras-applications/convnext/"

In [4]:
MODEL_CONFIGS = {
    "tiny": {
        "depths": [3, 3, 9, 3],
        "projection_dims": [96, 192, 384, 768],
        "default_size": 224,
    },
    "small": {
        "depths": [3, 3, 27, 3],
        "projection_dims": [96, 192, 384, 768],
        "default_size": 224,
    },
    "base": {
        "depths": [3, 3, 27, 3],
        "projection_dims": [128, 256, 512, 1024],
        "default_size": 224,
    },
    "large": {
        "depths": [3, 3, 27, 3],
        "projection_dims": [192, 384, 768, 1536],
        "default_size": 224,
    },
    "xlarge": {
        "depths": [3, 3, 27, 3],
        "projection_dims": [256, 512, 1024, 2048],
        "default_size": 224,
    },
}

In [5]:
WEIGHTS_HASHES = {
  "tiny":
    ("594e0f8c77df6cdf30d07e92f19d530921a53ff8301edd130c1bb9ad3dd6f25b",
      "278a5149e8c6e26f001051db5a8398f40f233994af2921a072bdca54389a9048"),
  "small":
    ("8f26cee79fea02bbbbdc721e7dd5d416562106b7768ec37459850c7b23f26cb2",
      "b0700a330b2e8bfa6862b61d333d3d8860e566ce2f9395a2165e506935bba547"),
  "base":
    ("2c08893b86245f4fc1d80f584faeb7431b23e895b03f71fc3d943a3489b60089",
      "a3d12bf8938796ca00721db89cb9e5a1af77f847401348adef16fab3611233d3"),
  "large":
    ("f17f979cc7cd23906a88f490ff1249d2b0dedc44a601f5700a0873afc1b0ad02",
      "74ff1e5f6eee45c62194aaee4a3347d5e2cf574caccad414b2b117b3ad110b32"),
  "xlarge":
    ("784b923c2db18fec883093b265c2a44c4fc401425f206a3ca036015290f401e8",
      "ba28c90cf6a4c64acfe0ca3af3ad920652c50f24668a3aaaf245a1c47e7bfe6f"),
}

## Set up ImageNet-1k labels

In [6]:
with open("imagenet_class_index.json", "r") as read_file:
    imagenet_labels = json.load(read_file)

MAPPING_DICT = {}
LABEL_NAMES = {}
for label_id in list(imagenet_labels.keys()):
    MAPPING_DICT[imagenet_labels[label_id][0]] = int(label_id)
    LABEL_NAMES[int(label_id)] = imagenet_labels[label_id][1]

In [7]:
all_val_paths = list(paths.list_images("val"))
all_val_labels = [MAPPING_DICT[x.split("/")[1]] for x in all_val_paths]

all_val_paths[:5], all_val_labels[:5]

(['val/n03000134/ILSVRC2012_val_00009432.JPEG',
  'val/n03000134/ILSVRC2012_val_00018410.JPEG',
  'val/n03000134/ILSVRC2012_val_00043280.JPEG',
  'val/n03000134/ILSVRC2012_val_00041208.JPEG',
  'val/n03000134/ILSVRC2012_val_00014205.JPEG'],
 [489, 489, 489, 489, 489])

## Preprocessing utilities

In [8]:
# Model already has a normalization layer inside.
def load_and_prepare(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (256, 256), method="bicubic")
    image = tf.image.central_crop(image, 0.875)
    return image, label

## Prepare `tf.data.Dataset`

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((all_val_paths, all_val_labels))
dataset = dataset.map(load_and_prepare, num_parallel_calls=AUTO).batch(BATCH_SIZE)
dataset = dataset.prefetch(AUTO)
dataset.element_spec

2022-05-02 09:05:46.601711: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 09:05:52.695772: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38414 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

## Initialize models and run eval

In [10]:
for model_name in MODEL_CONFIGS:
    config = MODEL_CONFIGS.get(model_name)
    model = convnext.ConvNeXt(
        **config,
        include_top=True
    )
    checkpoint_path = os.path.join(BASE_WEIGHTS_PATH, f"convnext_{model_name}.h5")
    print(f"Fetching checkpoint from {checkpoint_path}.")
    
    file_hash = WEIGHTS_HASHES.get(model_name)[0]
    weights_path = keras.utils.get_file(
        f"convnext_{model_name}.h5",
        checkpoint_path,
        cache_subdir="models",
        file_hash=file_hash
    )
    model.load_weights(weights_path)
    model.compile(metrics=["accuracy"])
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs_{model_name}")
    
    _, accuracy = model.evaluate(dataset, callbacks=[tb_callback])
    accuracy = round(accuracy * 100, 4)
    print(f"{model_name}: {accuracy}%.", file=open(f"{model_name}.txt", "w"))

Fetching checkpoint from https://storage.googleapis.com/convnext-tf/keras-applications/convnext/convnext_tiny.h5.
A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 594e0f8c77df6cdf30d07e92f19d530921a53ff8301edd130c1bb9ad3dd6f25b so we will re-download the data.
114589696/114579080 [==============================] - 1s 0us/step


2022-05-02 09:06:04.717526: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200
2022-05-02 09:06:10.428064: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


196/196 [==============================] - 89s 391ms/step - loss: 0.0000e+00 - accuracy: 0.8131
Fetching checkpoint from https://storage.googleapis.com/convnext-tf/keras-applications/convnext/convnext_small.h5.
A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 8f26cee79fea02bbbbdc721e7dd5d416562106b7768ec37459850c7b23f26cb2 so we will re-download the data.
196/196 [==============================] - 89s 428ms/step - loss: 0.0000e+00 - accuracy: 0.8239
Fetching checkpoint from https://storage.googleapis.com/convnext-tf/keras-applications/convnext/convnext_base.h5.
A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 2c08893b86245f4fc1d80f584faeb7431b23e895b03f71fc3d943a3489b60089 so we will re-download the data.
196/196 [==============================] - 117s 567ms/step - loss: 0.0000e+00 - accuracy: 0.8536
Fetching checkpoint from h